<a href="https://colab.research.google.com/github/sreemolgagarin/ML-project_2/blob/main/OSELM_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.utils.extmath import safe_sparse_dot
from scipy.linalg import pinv2,inv
from sklearn.utils import check_random_state
from sklearn.preprocessing import LabelBinarizer
from math import log
import numpy as np
import random

class OSElmClassifier():
    
    def __init__(self,n_hidden=20,threshold = 0.3,random_state=0,binarizer=LabelBinarizer(0, 1)):
        self.n_hidden = n_hidden
        self.binarizer = binarizer
        self.weights = None
        self.biases = None
        self.beta = None
        self.threshold = threshold
        self.random_state = random_state
        self.class_predictions=None
        self.H_cur = self.k_cur = self.B_cur = None
        self.B_prev = self.k_prev = None
        self.beta = None
        
    def compute_components(self,X):
        rs = check_random_state(self.random_state)
        n_features = X.shape[1]
        hw_size = (n_features, self.n_hidden)
        self.weights = rs.normal(size=hw_size)
        b_size = self.n_hidden
        self.biases = rs.normal(size=b_size)

    def compute_activation(self,X):
        _sigmoid = (lambda x: 1.0/(1.0 + np.exp(-x)))
        self.compute_components(X)
        weighted_sum= safe_sparse_dot(X,self.weights)+self.biases
        h_layer_acti = _sigmoid(weighted_sum)
        return h_layer_acti
    
    def initialize_param(self,H0,y):
        k0 = safe_sparse_dot(H0.T,H0)
        B0 =safe_sparse_dot(pinv2(H0),y)
        return k0, B0
    
    def update_beta(self,B_prev,k_prev,H_cur,y):
        temp = safe_sparse_dot(H_cur.T,H_cur)
        self.k_cur= k_prev+temp
        temp = inv(self.k_cur) 
        temp = safe_sparse_dot(temp, H_cur.T)
        temp = safe_sparse_dot(temp,(y-safe_sparse_dot(H_cur,B_prev)))
        self.B_cur = B_prev + temp
        self.beta = self.B_cur
        
    def CrossEntropy(self,yHat,y):
        loss = []
        for i in range (0,y.shape[0]):
            if y[i]==1:
                loss.append(-(log(yHat[i])))
            else:
                loss.append(-(log(1-yHat[i])))
        loss = np.array(loss)
        return loss
        
    def entropy(self,y_hat,y):
        
        
        
        
    def fit(self,X,y):
        y_bin = self.binarizer.fit_transform(y)
        self.H_cur = self.compute_activation(X)
        self.k_cur, self.B_cur = self.initialize_param(self.H_cur,y)
        self.k_prev, self.B_prev = self.k_cur, self.B_cur
        self.beta = self.B_cur
           
    def train(self,X,y):
        _sigmoid = (lambda x: 1.0/(1.0 + np.exp(-x)))
        self.H_cur = self.compute_activation(X)
        predicted_output = safe_sparse_dot(self.H_cur,self.B_cur)
        y_pred = _sigmoid(predicted_output)
        #loss = self.CrossEntropy(y_pred,y)
        outliers = np.where(np.logical_or(y_pred <= 0.4,y_pred >= 0.6))
        outliers = np.reshape(outliers,(-1))
        indices = np.random.choice(outliers,20)
        uncertain = np.where(np.logical_and(y_pred >0.4, y_pred<.6))
        uncertain = np.reshape(uncertain,(-1))
        indices = np.append(indices,np.random.choice(uncertain,50))
        indices = np.unique(indices)
        #indices = np.array(indices)
        #indices = np.reshape(indices,(-1))
        if any(i<.99 and i>=0.6 for i in y_pred):
            X_labeled,y_labeled = X[indices],y[indices]
            print(y_pred[indices],y[indices])
            self.H_cur = self.compute_activation(X_labeled)
            self.update_beta(self.B_prev,self.k_prev,self.H_cur,y_labeled)
            self.k_prev, self.B_prev = self.k_cur, self.B_cur
            return False
        else:
            print("True")
            return True
        
    def predict(self,X,y):
        h_layer_acti = self.compute_activation(X)
        predict=safe_sparse_dot(h_layer_acti,self.beta)
        self.class_predictions = self.binarizer.inverse_transform(predict)
        return self.class_predictions
    
    def regression_fit(self,X,y):
        
        self.H_cur = self.compute_activation(X)
        self.k_cur, self.B_cur = self.initialize_param(self.H_cur,y)
        self.k_prev, self.B_prev = self.k_cur, self.B_cur
        self.beta = self.B_cur
    
    def regression_predict(self,X,y):
        h_layer_acti = self.compute_activation(X)
        predict=safe_sparse_dot(h_layer_acti,self.beta)
        #self.class_predictions = self.binarizer.inverse_transform(predict)
        return predict
    
    def score(self,y):
        from sklearn.metrics import accuracy_score
        return accuracy_score(y,self.class_predictions) 
        
